## Simple Lambda

In [ ]:
region = "us-east-1"

In [ ]:
import json
import boto3
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda', region_name = region)

lambda_execution_role_name = "LambdaExecutionRole"
policy_name = "LogAccessPolicy"

response = iam_client.create_role(
    RoleName=lambda_execution_role_name,
    AssumeRolePolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }),
)

lambda_role_arn = response["Role"]["Arn"]

iam_client.put_role_policy(
    RoleName=lambda_execution_role_name,
    PolicyName=policy_name,
    PolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "logs:*",
                ],
                "Resource": "*"
            }
        ]
    })
)

In [ ]:
import io
import zipfile

lambda_function_name = "lambda_mcp_get_logs_lambda"
zip_buffer = io.BytesIO()
with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zf:
    zf.write('lambda_function.py')
zip_buffer.seek(0)

# wait for couple seconds for the role above to be created before creating lambda
response = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.13',
    Role=lambda_role_arn,
    Handler='lambda_function.lambda_handler',
    Code={
        'ZipFile': zip_buffer.read()
    },
    Timeout=60,
    Publish=True,
)

lambda_arn = response["FunctionArn"]


In [ ]:
print(lambda_arn)

## Create Strands Agent

In [ ]:
from strands.tools.mcp import MCPClient
from mcp import stdio_client, StdioServerParameters

# MCP Client Setting
lambda_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx",
            args=["awslabs.lambda-tool-mcp-server@latest"],
            env={
                # "AWS_PROFILE": "your-aws-profile",
                "AWS_REGION": region, # same region as where we have created lambda
                # "FUNCTION_PREFIX": "your-function-prefix",
                "FUNCTION_LIST": lambda_function_name, # comman seperated function names, ie: your-first-function, your-second-function
                # "FUNCTION_TAG_KEY": "your-tag-key",
                # "FUNCTION_TAG_VALUE": "your-tag-value",
                # "FUNCTION_INPUT_SCHEMA_ARN_TAG_KEY": "your-function-tag-for-input-schema"
            }
        )
    )
)
lambda_mcp_client.start()

In [ ]:
# import boto3
from strands import Agent
from strands.models import BedrockModel

bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name=region,
)

agent = Agent(
    model=bedrock_model,
    tools=lambda_mcp_client.list_tools_sync()
)

## Test it out

In [ ]:
agent.tool_registry.get_all_tools_config()

# {'lambda_mcp_get_logs_lambda': {'inputSchema': {'json': {'properties': {'parameters': {'additionalProperties': True,
#       'title': 'Parameters',
#       'type': 'object',
#       'description': 'Property parameters'}},
#     'required': ['parameters'],
#     'title': 'lambda_functionArguments',
#     'type': 'object'}},
#   'name': 'lambda_mcp_get_logs_lambda',
#   'description': 'Tool which performs lambda_mcp_get_logs_lambda'}}

In [ ]:
# without any function parameters required
result = agent("get my aws logs for me")
print(result.message["content"])
# I'll help you get your AWS logs. Let me retrieve them for you.
# Tool #1: lambda_mcp_get_logs_lambda
# Here are your AWS CloudWatch log groups. I can see you have quite a collection of logs from various AWS services! Here's a summary of what I found:

# ## Log Groups Overview:

# **Bedrock Agent Core Logs:**
# - Runtime demo logs
# - Runtime with memory logs

# **CloudFront Function Logs:**
# - API test functions
# - Header update functions
# - Print API test functions
# ...

In [ ]:
# with function parameters required
result = agent("get my aws lambda logs for me")

# I'll get your AWS Lambda logs specifically. Let me retrieve them for you.
# Tool #2: lambda_mcp_get_logs_lambda
# Here are your AWS Lambda function logs specifically. I've filtered out just the Lambda-related log groups from your CloudWatch logs:

# ## AWS Lambda Log Groups:

# ### **Business/Application Functions:**
# 1. **...** - 41.7KB of logs
# 2. **...** - 69.7KB (731-day retention)
# ...

In [ ]:
tool_uses_messages = list(filter(lambda x: x["role"] == "assistant" and any(content.get("toolUse") is not None for content in x["content"]), agent.messages))
for message in tool_uses_messages:
    print("----------")
    contents = message["content"]
    for content in contents:
        if content.get("toolUse") is not None:
            tool_use = content.get("toolUse")
            print(f"tool name: {tool_use["name"]}")
            print(f"input: {tool_use["input"]}")

# ----------
# tool name: lambda_mcp_get_logs_lambda
# input: {'parameters': {}}
# ----------
# tool name: lambda_mcp_get_logs_lambda
# input: {'parameters': {'service': 'lambda'}} # sepcifying not-existing key!

## Delete resources

In [ ]:

# delete lambda
response = lambda_client.delete_function(
    FunctionName=lambda_function_name,
)

# delete role policy and role
iam_client.delete_role_policy(
    RoleName=lambda_execution_role_name,
    PolicyName=policy_name
)

iam_client.delete_role(
    RoleName = lambda_execution_role_name
)
